In [1]:
import pandas as pd
import numpy as np
import io, os, sys
import glob


In [ ]:
    def centered_point_clouds(self, files):

        train_set = []
        
        for filename in files:
            data_set = pd.read_csv(filename, skiprows = [0,1],
                    header = None, names = ['x', 'y', 'z', 'c'], sep = " ")
            data_set = data_set[data_set.isnull().any(axis = 1)]
            data_set = data_set.iloc[:, :-1]
            data_set = data_set.to_numpy() 

            max_min = (np.abs(np.max(data_set, axis = 0)) - np.abs(np.min(data_set, axis = 0))) / 2


            Canonical_ST = np.zeros([1,3])

            Canonical_ST[0,0] = max_min[0]
            Canonical_ST[0,1] = max_min[1]
            Canonical_ST[0,2] = max_min[2]

            centered_point_clouds = data_set - Canonical_ST

            train_set.append(centered_point_clouds)

        return train_set


In [76]:
class data_clean():
    
    
    
    """
    centered the point clouds and make a total train/test set
    """
    def __init__(self, root_dir):
        """
        Args:
            files
            root_dir (string): Directory with the ModelNet.

        """
        self.root_dir = root_dir
    

    def centered_point_clouds(self, files):

        train_set = []
        
        for filename in files:
            data_set = pd.read_csv(filename, skiprows = [0,1],
                    header = None, names = ['x', 'y', 'z', 'c'], sep = " ")
            data_set = data_set[data_set.isnull().any(axis = 1)]
            data_set = data_set.iloc[:, :-1]
            data_set = data_set.to_numpy() 

            max_min = (np.abs(np.max(data_set, axis = 0)) - np.abs(np.min(data_set, axis = 0))) / 2


            Canonical_ST = np.zeros([1,3])

            Canonical_ST[0,0] = max_min[0]
            Canonical_ST[0,1] = max_min[1]
            Canonical_ST[0,2] = max_min[2]

            centered_point_clouds = data_set - Canonical_ST

            train_set.append(centered_point_clouds)

        return train_set


    def total_set(self):

        total_train_set = []
        total_test_set = []
        empty_total_train_df= pd.DataFrame()
        empty_total_test_df= pd.DataFrame()
        class_list = os.listdir(self.root_dir) # to find how many classes are, of course 40.....
        

        for i in range(len(class_list)):
            
            train_files = glob.glob(self.root_dir + class_list[i] + '/train/*.off')
            test_files = glob.glob(self.root_dir + class_list[i] + '/test/*.off')

            train = self.centered_point_clouds(train_files)
            test = self.centered_point_clouds(test_files)
            
            train_df = pd.DataFrame(train)
            train_df['class'] = i
            
            test_df = pd.DataFrame(test)
            test_df['class'] = i

            
            empty_total_train_df = empty_total_train_df.append(train_df)
            empty_total_test_df = empty_total_test_df.append(test_df)
            
        return empty_total_train_df, empty_total_test_df
    


In [77]:
clean_data = data_clean('./ModelNet40/')

In [78]:
total_train_set, total_test_set = clean_data.total_set()

In [95]:
train_df = total_train_set.to_csv("train_df.csv", index = False)

In [96]:
test_df = total_test_set.to_csv("test_df.csv", index =False)

In [103]:
train_df = pd.read_csv('./train_df.csv')

In [104]:
test_df = pd.read_csv('./test_df.csv')

In [106]:
print("how many classes? ", len(train_df), "and how many train set in the first class? " , len(test_df))

how many classes?  9843 and how many train set in the first class?  2468
